In [ ]:
#before running this please change the RUNTIME to GPU (Runtime -> Change runtime type -> set harware accelarotor as GPU)
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import libraries
!pip3 install face_recognition

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=c10776ee5c1c5887ba8c0c0d963659f93f6579dc5e01b744df7b0e7898dc1f1c
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
#import libraries
import torch
from torch.autograd import Variable
import time
import os
import sys
import os
from torch import nn
from torchvision import models

In [ ]:
#Model with feature visualization
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c,  h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(x_lstm[:,-1,:]))

In [ ]:
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
sm = nn.Softmax()
inv_normalize =  transforms.Normalize(mean=-1*np.divide(mean,std),std=np.divide([1,1,1],std))
def im_convert(tensor):
    """ Display a tensor as an image. """
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze()
    image = inv_normalize(image)
    image = image.numpy()
    image = image.transpose(1,2,0)
    image = image.clip(0, 1)
    cv2.imwrite('./2.png',image*255)
    return image

def predict(model,img,path = './'):
  fmap,logits = model(img.to('cuda'))
  params = list(model.parameters())
  weight_softmax = model.linear1.weight.detach().cpu().numpy()
  print('Before Softmax')
  print(logits)
  logits = sm(logits)
  print('After Softmax')
  print(logits)
  # _,prediction = torch.max(logits,1)
  predictionVal = logits[0][1]
  print(predictionVal)
  # confidence = logits[:,int(prediction.item())].item()*100
  # print('confidence of prediction:',logits[:,int(prediction.item())].item()*100)
  confidence = predictionVal*100
  print('confidence of prediction: ',predictionVal*100)
  idx = np.argmax(logits.detach().cpu().numpy())
  bz, nc, h, w = fmap.shape
  out = np.dot(fmap[-1].detach().cpu().numpy().reshape((nc, h*w)).T,weight_softmax[idx,:].T)
  predict = out.reshape(h,w)
  predict = predict - np.min(predict)
  predict_img = predict / np.max(predict)
  predict_img = np.uint8(255*predict_img)
  out = cv2.resize(predict_img, (im_size,im_size))
  heatmap = cv2.applyColorMap(out, cv2.COLORMAP_JET)
  img = im_convert(img[:,-1,:,:,:])
  result = heatmap * 0.5 + img*0.8*255
  cv2.imwrite('/content/1.png',result)
  result1 = heatmap * 0.5/255 + img*0.8
  r,g,b = cv2.split(result1)
  result1 = cv2.merge((r,g,b))
  plt.imshow(result1)
  plt.show()
  # return [int(prediction.item()),confidence]
  return confidence
#img = train_data[100][0].unsqueeze(0)
#predict(model,img)

In [ ]:
#!pip3 install face_recognition
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
class validation_dataset(Dataset):
    def __init__(self,video_names,sequence_length = 60,transform = None):
        self.video_names = video_names
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self,idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100/self.count)
        first_frame = np.random.randint(0,a)
        for i,frame in enumerate(self.frame_extract(video_path)):
            #if(i % a == first_frame):
            faces = face_recognition.face_locations(frame)
            try:
              top,right,bottom,left = faces[0]
              frame = frame[top:bottom,left:right,:]
            except:
              pass
            frames.append(self.transform(frame))
            if(len(frames) == self.count):
              break
        print("no of frames",len(frames))
        frames = torch.stack(frames)
        frames = frames[:self.count]
        return frames.unsqueeze(0)
    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path)
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image
def im_plot(tensor):
    image = tensor.cpu().numpy().transpose(1,2,0)
    b,g,r = cv2.split(image)
    image = cv2.merge((r,g,b))
    image = image*[0.22803, 0.22145, 0.216989] +  [0.43216, 0.394666, 0.37645]
    image = image*255.0
    plt.imshow(image.astype(int))
    plt.show()

In [ ]:
from torchsummary import summary

#Code for making prediction
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
# path_to_videos = ['/content/drive/My Drive/Balanced_Face_only_data/aagfhgtpmv.mp4',
#                                    '/content/drive/My Drive/Balanced_Face_only_data/aczrgyricp.mp4',
#                                    '/content/drive/My Drive/Balanced_Face_only_data/agdkmztvby.mp4',
#                                    '/content/drive/My Drive/Balanced_Face_only_data/abarnvbtwb.mp4']

# path_to_videos = ['/content/drive/My Drive/Youtube_Face_only_data/000_003.mp4',
#                   '/content/drive/My Drive/Youtube_Face_only_data/000.mp4',
#                   '/content/drive/My Drive/Youtube_Face_only_data/002_006.mp4',
#                   '/content/drive/My Drive/Youtube_Face_only_data/002.mp4'


# ]

# path_to_videos= ["/content/drive/My Drive/DFDC_REAL_Face_only_data/aabqyygbaa.mp4"]

# path_to_videos= ["/content/drive/MyDrive/VideoTestTry/Fake1.mp4", "/content/drive/MyDrive/VideoTestTry/Real5SecVideo.mp4", "/content/drive/MyDrive/VideoTestTry/Fake10Sec.mp4"]
# # path_to_videos= ["/content/drive/MyDrive/VideoTestTry/Fake5SecVideo.mp4", "/content/drive/MyDrive/VideoTestTry/Real5SecVideo.mp4"]
# video_dataset = validation_dataset(path_to_videos,sequence_length = 60,transform = train_transforms)

model = Model(2).cuda()
path_to_model = '/content/drive/MyDrive/model_90_acc_60_frames_final_data.pt'
model.load_state_dict(torch.load(path_to_model))
model.eval()

# for i in range(0,len(path_to_videos)):
#   print(path_to_videos[i])
#   predictConfidence = predict(model,video_dataset[i],'./')
#   # print(predictConfidence[0])
#   # print('---')
#   # print(predictConfidence[1])
#   # if predictConfidence[0] == 1:
#   #   print("REAL")
#   # else:
#   #   print("FAKE")
#   if predictConfidence > 60:
#     print(f"With {predictConfidence} confidence it is a REAL video")
#   else:
#     tempX = 100 - predictConfidence
#     print(f"With {tempX} confidence it is a Deep Faked video")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:02<00:00, 48.8MB/s]


Model(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [ ]:
def fakeOrNot(VideoFileName):
  full_path_to_videos = "/content/drive/MyDrive/" + VideoFileName + ".mp4"
  path_to_videos_list = [full_path_to_videos]
  print(path_to_videos_list[0])
  video_dataset = validation_dataset(path_to_videos_list,sequence_length = 60,transform = train_transforms)
  predictConfidence = predict(model,video_dataset[0],'./')
  # if predictConfidence > 60:
  #   return (f"With {predictConfidence} confidence it is a REAL video")
  # else:
  #   tempX = 100 - predictConfidence
  #   return (f"With {tempX} confidence it is a Deep Faked video")
  if predictConfidence > 65:
    return (f"It is a REAL video.")
  else:
    tempX = 100 - predictConfidence
    return (f"It is Deep Faked video.")

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-pl

In [ ]:
# Path input Working

# def video_identity(videoName):
#   return fakeOrNot(videoName)
import gradio as gr

demo = gr.Interface(fn=fakeOrNot, inputs="text", outputs="text", title="Deep Fake Detection", description="Enter the video name to check it is a deepfake video or not.")
demo.launch(debug=True)

# path - /content/drive/MyDrive/LocalFiveSecRealVideo.mp4

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/content/drive/MyDrive/Fake1.mp4


<ipython-input-5-2b32dff8c7b9>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = sm(logits)


no of frames 60
Before Softmax
tensor([[-0.1114,  0.2634]], device='cuda:0', grad_fn=<AddmmBackward0>)
After Softmax
tensor([[0.4074, 0.5926]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor(0.5926, device='cuda:0', grad_fn=<SelectBackward0>)
confidence of prediction:  tensor(59.2610, device='cuda:0', grad_fn=<MulBackward0>)
Keyboard interruption in main thread... closing server.


In [ ]:
import cv2
import gradio as gr

def save_video(video_file):
    # Create a VideoCapture object to read the video
    cap = cv2.VideoCapture(video_file)

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = '/content/drive/My Drive/output_video.mp4'
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (640, 480))

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Write the frame to the output video file
        out.write(frame)

        # Display the resulting frame
        cv2.imshow('Video', frame)

        # Press 'q' to stop the video capturing
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture and VideoWriter objects
    cap.release()
    out.release()

    # Destroy all the windows
    cv2.destroyAllWindows()

    return output_path

iface = gr.Interface(fn=save_video,
                     inputs="file",
                     outputs="text",
                     interpretation="default",
                     title="Video Saving Interface",
                     description="Select a video file to save to Google Drive.")


In [ ]:
iface.launch(debug=True)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1077, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    result = context.run(func, *args)
  File "<ipython-input-13-d71bf3dedc56>", line 6, in save_video
    cap = cv2.VideoCapture(video_file)
cv2.error: OpenCV(4.7.

Keyboard interruption in main thread... closing server.


In [ ]:
!pip install streamlit
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.9 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=97d8e7150fb9c8e77d500b4cbcaadbdd683d52cbce374360e608afeb3aa3631e
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
 

In [ ]:
import streamlit as st
import cv2
from pyngrok import ngrok

# # Streamlit app
# def main():
#     # Set title and description
st.title("Video Fake Detection with CV2 and DL")
st.write("Select a video file for analysis.")

# File uploader
video_file = st.file_uploader("Upload a video file", type=["mp4", "avi"])

# Check if a video file is uploaded
if video_file is not None:
  # Display the video
  st.video(video_file)

  # Process the video and get the fake detection result
  text_output = fakeOrNot(video_file)

  # Display the fake detection result
  st.write("Fake Detection Result:")
  st.write(text_output)

# Run the Streamlit app
# if __name__ == "__main__":
#     # Start a new Streamlit app
#     !npx localtunnel --port 8501 & curl ipv4.icanhazip.com
#     main()


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.599s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & curl ipv4.icanhazip.com

34.87.182.19


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.072s
your url is: https://shy-hounds-guess.loca.lt
/root/.npm/_npx/11745/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:41707 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/11745/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorCloseNT (internal/streams/destroy.js:74:3)
    at processTicksAndRejections (internal/process/task_queues.js:80:21)


In [ ]:
import streamlit as st
import cv2
from pyngrok import ngrok

# Function to check if a video is real or deep faked
def fakeOrNot(full_path_to_videos):
    # full_path_to_videos = "/content/drive/MyDrive/" + videoName + ".mp4"
    path_to_videos_list = [full_path_to_videos]
    print(path_to_videos_list[0])
    video_dataset = validation_dataset(path_to_videos_list, sequence_length=60, transform=train_transforms)
    predictConfidence = predict(model, video_dataset[0], './')
    if predictConfidence > 60:
        return f"With {predictConfidence} confidence, it is a REAL video"
    else:
        tempX = 100 - predictConfidence
        return f"With {tempX} confidence, it is a Deep Faked video"

# Streamlit app
def main():
    # Set title and description
    st.title("Video Fake Detection with CV2 and DL")
    st.write("Select a video file for analysis.")

    # File uploader
    video_file = st.file_uploader("Upload a video file", type=["mp4", "avi"])

    # Check if a video file is uploaded
    if video_file is not None:
        # Display the video
        st.video(video_file)

        # Process the video and get the fake detection result
        text_output = fakeOrNot(video_file)

        # Display the fake detection result
        st.write("Fake Detection Result:")
        st.write(text_output)

# Run the Streamlit app
if __name__ == "__main__":
    # Start the Streamlit app
    public_url = ngrok.connect(port='8501')
    print('Public URL:', public_url)
    main()


PyngrokNgrokHTTPError: ignored

In [ ]:
# def fakeOrNot(path_to_videos):
#   # for i in range(0,len(path_to_videos)):
#     # print(path_to_videos[0])
#     path_to_videos_list = [path_to_videos]
#     video_dataset = validation_dataset(path_to_videos_list,sequence_length = 20,transform = train_transforms)
#     prediction = predict(model,video_dataset[0],'./')
#     if prediction[0] == 1:
#       return "REAL"
#       # print("REAL")
#     else:
#       # print("FAKE")
#       return "FAKE"

In [ ]:
# import gradio as gr
# import tempfile

# def process_video_interface(video):
#     print(video)
#     # Convert the string representation of video to bytes
#     video_bytes = video.encode('utf-8')

#     # Save the video bytes to a temporary file
#     with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as tempNewNew:
#         tempNewNew.write(video_bytes)
#         tempNewNew.flush()
#         temp_path = tempNewNew.name

#     full_path = os.path.join(os.getcwd(), temp_path)
#     print(full_path)
#     return fakeOrNot(full_path)

# demo = gr.Interface(fn=process_video_interface, inputs="video", outputs="text")
# demo.launch(share=True, debug=True)

In [ ]:
# #  Save video in temp Working

# import gradio as gr
# import tempfile

# def save_video_interface(video):
#     # Convert the string representation of the video to bytes
#     video_bytes = video.encode('utf-8')

#     # Save the video bytes to a temporary file
#     with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as temp_file:
#         temp_file.write(video_bytes)
#         temp_file_path = temp_file.name

#     return f"Video saved at: {temp_file_path}"

# # Define the Gradio interface
# video_input = gr.inputs.Video(label="Select a video")
# save_output = gr.outputs.Textbox(label="Save status")

# interface = gr.Interface(
#     fn=save_video_interface,
#     inputs=video_input,
#     outputs=save_output,
#     title="Video Saver",
#     description="Select a video and save it to a temporary file.",
#     theme="default"
# )

# # Launch the Gradio interface
# interface.launch()


In [ ]:
# #  Save video in content Working

# import gradio as gr
# import tempfile
# import os

# def save_video_interface(video):
#     # Convert the string representation of the video to bytes
#     video_bytes = video.encode('utf-8')

#     # Save the video bytes to a temporary file in /content directory
#     with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False, dir='/content/drive/MyDrive/predictVideoStore') as temp_file:
#         temp_file.write(video_bytes)
#         temp_file_path = temp_file.name

#     # pleaseWork = fakeOrNot(temp_file_path)

#     return f"{temp_file_path}"

# # Define the Gradio interface
# video_input = gr.inputs.Video(label="Select a video")
# save_output = gr.outputs.Textbox(label="Save status")

# interface = gr.Interface(
#     fn=save_video_interface,
#     inputs=video_input,
#     outputs=save_output,
#     title="Video Saver",
#     description="Select a video and save it to /content directory.",
#     theme="default"
# )

# # Launch the Gradio interface
# interface.launch(debug = True)


In [ ]:
# import gradio as gr
# import tempfile
# import cv2

# def video_to_text(video):
#     # Store the video temporarily
#     temp_video = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)
#     temp_video_path = temp_video.name
#     temp_video.close()
#     video.save(temp_video_path)

#     # Process the video and get text output
#     text_output = fakeOrNot([temp_video_path])

#     # Return the text output
#     return text_output

# # Define the input and output interfaces
# video_input = gr.inputs.Video(label="Upload a video")
# text_output = gr.outputs.Textbox(label="Text Output")

# # Create the Gradio interface
# gr.Interface(fn=video_to_text, inputs=video_input, outputs=text_output, cleanup = True).launch()

In [ ]:
# import gradio as gr
# import tempfile
# import cv2

# def video_to_text(file):
#     # Process the video and get text output
#     text_output = fakeOrNot([file])

#     # Return the text output
#     return text_output

# # Define the input and output interfaces
# file_input = gr.inputs.File(label="Upload a video")
# text_output = gr.outputs.Textbox(label="Text Output")

# # Create the Gradio interface
# gr.Interface(fn=video_to_text, inputs=file_input, outputs=text_output).launch()

In [ ]:
# import gradio as gr
# import tempfile
# import cv2

# def video_to_display(file):
#     # Define the variable video and assign it a value
#     video = cv2.VideoCapture(file)

#     # Store the video temporarily
#     temp_video = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)
#     temp_video_path = temp_video.name
#     temp_video.close()
#     video.save(temp_video_path)

#     # Display the video in Colab
#     gr.outputs.Video(temp_video_path, label="Video Output")

# # Define the input and output interfaces
# file_input = gr.components.File(label="Upload a video")

# # Create the Gradio interface
# gr.Interface(fn=video_to_display, inputs=file_input, outputs=gr.outputs.Video()).launch(debug=True)

In [ ]:
# net = model
# modules = [module for module in net.modules()]
# params = [param.shape for param in net.parameters()]

# # Print Model Summary
# print(modules[0])
# total_params=0
# for i in range(1,len(modules)):
#    j = 2*i
#    param = (params[j-2][1]*params[j-2][0])+params[j-1][0]
#    total_params += param
#    print("Layer",i,"->\t",end="")
#    print("Weights:", params[j-2][0],"x",params[j-2][1],
#          "\tBias: ",params[j-1][0], "\tParameters: ", param)
# print("\nTotal Params: ", total_params)

In [ ]:
# # Displaying video

# tryPath = "/content/drive/My Drive/LocalFiveSecRealVideo.mp4"

# from IPython.display import HTML
# from base64 import b64encode

# def show_video(video_path, video_width = 600):

#   video_file = open(video_path, "r+b").read()

#   video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
#   return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

# show_video(tryPath)

In [ ]:
from torch.nn.modules.module import _addindent
import torch
import numpy as np
def torch_summarize(model, show_weights=True, show_parameters=True):
    """Summarizes torch model by showing trainable parameters and weights."""
    tmpstr = model.__class__.__name__ + ' (\n'
    for key, module in model._modules.items():
        # if it contains layers let call it recursively to get params and weights
        if type(module) in [
            torch.nn.modules.container.Container,
            torch.nn.modules.container.Sequential
        ]:
            modstr = torch_summarize(module)
        else:
            modstr = module.__repr__()
        modstr = _addindent(modstr, 2)

        params = sum([np.prod(p.size()) for p in module.parameters()])
        weights = tuple([tuple(p.size()) for p in module.parameters()])

        tmpstr += '  (' + key + '): ' + modstr
        if show_weights:
            tmpstr += ', weights={}'.format(weights)
        if show_parameters:
            tmpstr +=  ', parameters={}'.format(params)
        tmpstr += '\n'

    tmpstr = tmpstr + ')'
    return tmpstr

# Test

print(torch_summarize(model))

Model (
  (model): Sequential (
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False), weights=((64, 3, 7, 7),), parameters=9408
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), weights=((64,), (64,)), parameters=128
    (2): ReLU(inplace=True), weights=(), parameters=0
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False), weights=(), parameters=0
    (4): Sequential (
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-0